In [1]:
from random import shuffle
import glob
import cv2
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
shuffle_data = True  # shuffle the addresses before saving
hdf5_path = '/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/dataset_completo.hdf5'  # address to where you want to save the hdf5 file
dataset_train_path = '/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/completo/*.txt'


In [3]:
addrs = glob.glob(dataset_train_path)
labels = []
#extraer el primer numero del nombre del archivo, para guardar en un arreglo
for file in addrs:
    m = re.search('completo/(.+?)_', file)
    #print (m.group(1))
    labels.append(int(m.group(1)))

In [4]:
labl = 60
print (labels[labl], addrs[labl])

10 /home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/completo/10_220_12_2.txt


In [5]:
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)

In [6]:
labl = 60
print (labels[labl], addrs[labl])
len(labels)

3 /home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/completo/3_780_1_6.txt


13455

In [7]:
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

In [8]:
import numpy as np
import h5py

/home/miguelmg/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
data_order = 'tf'  # 'th' for Theano, 'tf' for Tensorflow
# check the order of data and chose proper data shape to save images
if data_order == 'th': # it is not used
    train_shape = (len(train_addrs), 3, 224, 224)
elif data_order == 'tf':
    train_shape = (len(train_addrs), 32, 32, 32)
    test_shape = (len(test_addrs), 32, 32, 32)
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')

#hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)

hdf5_file.create_dataset("train_img", train_shape, np.float32)
hdf5_file.create_dataset("train_labels", (len(train_addrs),), np.int8)
hdf5_file["train_labels"][...] = train_labels

hdf5_file.create_dataset("test_img", test_shape, np.float32)
hdf5_file.create_dataset("test_labels", (len(test_addrs),), np.int8)
hdf5_file["test_labels"][...] = test_labels

In [10]:
# loop over train addresses
for i in range(len(train_addrs)):
    # print how many images are saved every 1000 images
    if i % 100 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = np.loadtxt(addr, dtype = float)
    img = img.reshape(32,32,32)

    # save the image
    hdf5_file["train_img"][i, ...] = img[None]

# loop over test addresses
for i in range(len(test_addrs)):
    # print how many images are saved every 1000 images
    if i % 100 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = test_addrs[i]
    img = np.loadtxt(addr, dtype = float)
    img = img.reshape(32,32,32)

    # save the image 
    hdf5_file["test_img"][i, ...] = img[None]
    
    
hdf5_file.close()

Train data: 100/10764
Train data: 200/10764
Train data: 300/10764
Train data: 400/10764
Train data: 500/10764
Train data: 600/10764
Train data: 700/10764
Train data: 800/10764
Train data: 900/10764
Train data: 1000/10764
Train data: 1100/10764
Train data: 1200/10764
Train data: 1300/10764
Train data: 1400/10764
Train data: 1500/10764
Train data: 1600/10764
Train data: 1700/10764
Train data: 1800/10764
Train data: 1900/10764
Train data: 2000/10764
Train data: 2100/10764
Train data: 2200/10764
Train data: 2300/10764
Train data: 2400/10764
Train data: 2500/10764
Train data: 2600/10764
Train data: 2700/10764
Train data: 2800/10764
Train data: 2900/10764
Train data: 3000/10764
Train data: 3100/10764
Train data: 3200/10764
Train data: 3300/10764
Train data: 3400/10764
Train data: 3500/10764
Train data: 3600/10764
Train data: 3700/10764
Train data: 3800/10764
Train data: 3900/10764
Train data: 4000/10764
Train data: 4100/10764
Train data: 4200/10764
Train data: 4300/10764
Train data: 4400/107

In [ ]:
import h5py
import numpy as np
hdf5_path = '/home/miguelmg/Documents/CIDETEC/semestre 2/vision 3d/proyecto/6d pose/hinterstoisser/nubes/dataset/dataset_completo.hdf5'  # address to where you want to save the hdf5 file
#subtract_mean = False
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")
# subtract the training mean
#if subtract_mean:
 #   mm = hdf5_file["train_mean"][0, ...]
 #   mm = mm[np.newaxis, ...]
# Total number of samples
data_num = hdf5_file["train_img"][0]
lbl = hdf5_file["train_labels"][0]
print (data_num.shape)
print (lbl)
#plt.imshow(data_num[:][:][20])